In [ ]:
import requests

response=requests.post(
    "http://localhost:8002/info/invoke",
    json={'input':{'dish':"chole"}})

facts = response.json()['output']['content']
print(response.json()['output']['content'])

In [ ]:
import re

# facts = """
# Nutritional Value:
# Calories: 210 (10%)
# Protein: 10g (15%)
# Carbohydrates: 35g (12%)
# Fats: 9g (13%)
# Fiber: 5g (20%)
# Sugars: 10g (11%)
# Cholesterol: 50mg (17%)
# Sodium: 600mg (26%)
# Iron: 3mg (17%)
# Calcium: 100mg (10%)

# Approximate Serving: 1 cup
# """

# Define a regular expression pattern to match each line

x=facts.split('Approximate Serving:')
serving = x[1].split('\n')[0].strip()
nutritional_values = x[0].split('Nutritional Value:')[1][1:]
nutrients = nutritional_values.split('\n')
nutritional_data = {}
for nutrient in nutrients:
    if nutrient:
        nutritional_data[nutrient.split(':')[0].strip()] = {'value':nutrient.split(':')[1].strip().split(' ')[0].strip(),'percentage':nutrient.split(':')[1].strip().split(' ')[1][1:-1].strip()}


In [ ]:
def get_nutritional_data(facts):
    x=facts.split('Approximate Serving:')
    serving = x[1].split('\n')[0].strip()
    nutritional_values = x[0].split('Nutritional Value:')[1][1:]
    nutrients = nutritional_values.split('\n')
    nutritional_data = {}
    for nutrient in nutrients:
        if nutrient:
            nutritional_data[nutrient.split(':')[0].strip()] = {'value':nutrient.split(':')[1].strip().split(' ')[0].strip(),'percentage':nutrient.split(':')[1].strip().split(' ')[1][1:-1].strip()}
    return nutritional_data,serving

In [ ]:
# nutritional_data to json
import json
nutritional_data_json = json.dumps(nutritional_data)
print(nutritional_data_json)
# Print the Calories value from the nutritional_data_json
print(json.loads(nutritional_data_json)['Calories']['value'])

In [ ]:
# Create a fastapi app with a single endpoint /nutritional_info post method that takes a json input with a key dish and returns the nutritional data and serving size of the dish

from fastapi import FastAPI
app = FastAPI()

@app.post("/nutritional_info")
async def get_nutritional_info(data: dict):
    dish = data['dish']
    response=requests.post(
        "http://localhost:8002/info/invoke",
        json={'input':{'dish':dish}})
    facts = response.json()['output']['content']
    nutritional_data,serving = get_nutritional_data(facts)
    # Convert the nutritional_data to json for easy parsing with javascript
    nutritional_data = json.dumps(nutritional_data)
    return {'nutritional_data':nutritional_data,'serving':serving}

# Run the app using uvicorn
import uvicorn
uvicorn.run(app, host="localhost", port=8001)